In [360]:
# Import data management libraries
from urbs import load
import pandas as pd
import numpy as np

In [361]:
year = 2020

In [362]:
# Import Excel file for reference and structure
xls = pd.ExcelFile(f'myopic/input/input_bayern_{year}.xlsx')

df = {}
for sheet in xls.sheet_names:
    df[sheet] = xls.parse(sheet_name = sheet, index_col = 0)

In [363]:
# Import longtable with timeseries information
ts = pd.read_excel("./myopic/input/longtable.xlsx")

In [364]:
# Update parameters on the Global tab
df['Global'].at['Support timeframe', 'value'] = year+5
df['Global'].at['CO2 limit', 'value'] = ts.loc[(ts['Sheet'] == 'Global') & (ts['Parameter'] == 'CO2 limit')][year+5] * 1e6

In [365]:
df['Process'].head()

,Process,inst-cap,lifetime,cap-lo,cap-up,max-grad,min-fraction,inv-cost,fix-cost,var-cost,startup-cost,wacc,depreciation,area-per-cap
Site,,,,,,,,,,,,,,
Bayern,Hydro plant,1981,10,0,1981.0,1,0,2208000,22080.000,0.00,0,0.07,60,NaN
Bayern,Onshore wind park,2533,25,0,inf,1,0,1118770,13984.625,1.50,0,0.07,25,NaN
Bayern,Photovoltaics,14035,25,0,inf,1,0,733470,11588.826,0.01,0,0.07,25,NaN
Bayern,Biomass plant,1848,30,0,1848.0,1,0,2209000,103823.000,0.00,0,0.07,30,NaN
Bayern,Geothermal plant,46,40,0,46.0,1,0,3392000,80051.200,0.00,0,0.07,40,NaN


In [369]:
# Update parameters on the Commodity, Process, and Storage tabs
for sheet in ['Commodity', 'Process', 'Storage']:
    # Iterate on the uniques parameter values per sheet to be updated
    for param in ts.loc[(ts['Sheet'] == sheet)]['Parameter'].unique():
        # Iterate on the IDs / technologies
        for idx in ts.loc[(ts['Sheet'] == sheet) & (ts['Parameter'] == param)]['ID'].unique():

            # Update output table with the base assumption and previous outputs
            #df[sheet].loc[df[sheet][sheet] == idx, param] = ts.loc[(ts['Sheet'] == sheet) & (ts['Parameter'] == 'param') & (ts['ID'] == idx)][year+5]

SyntaxError: unexpected EOF while parsing (<ipython-input-369-04ea6f230eab>, line 9)

In [ ]:
df['Process'].loc[df['Process']['Process'] == 'Hydro plant', 'inv-cost']

Site
Bayern   NaN
Name: inv-cost, dtype: float64

In [ ]:
df[sheet].loc[df[sheet][sheet] == idx, param]

Series([], Name: var-cost-c, dtype: float64)

In [ ]:
df['Process'].head()

,Process,inst-cap,lifetime,cap-lo,cap-up,max-grad,min-fraction,inv-cost,fix-cost,var-cost,startup-cost,wacc,depreciation,area-per-cap
Site,,,,,,,,,,,,,,
Bayern,Hydro plant,NaN,10,0,NaN,1,0,NaN,NaN,NaN,0,0.07,60,NaN
Bayern,Onshore wind park,NaN,25,0,NaN,1,0,NaN,NaN,NaN,0,0.07,25,NaN
Bayern,Photovoltaics,NaN,25,0,NaN,1,0,NaN,NaN,NaN,0,0.07,25,NaN
Bayern,Biomass plant,NaN,30,0,NaN,1,0,NaN,NaN,NaN,0,0.07,30,NaN
Bayern,Geothermal plant,NaN,40,0,NaN,1,0,NaN,NaN,NaN,0,0.07,40,NaN


In [ ]:
df['Demand'].head()

,Bayern.Space heating,Bayern.Process heating A,Bayern.Process heating B,Bayern.Process heating C,Bayern.Electric charging,Bayern.Elec_Rest
t,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,67.518741,4108.797002,2079.087724,1414.255932,201.744361,7293.329275
2,175.600523,4109.507691,2018.985616,1414.255932,195.912351,7293.329275
3,853.152823,4111.896924,1990.047564,1353.411762,193.104346,6979.555403
4,1143.413160,4117.974027,1985.595556,1301.471618,192.672345,6711.699659


In [ ]:
# Update the Demand tab

# Read profiles
profiles = pd.read_excel("./myopic/input/profiles.xlsx")

for col in df['Demand'].columns:
    comm = col.split('.')[1]
    df['Demand'].loc[:, col] = float(ts.loc[(ts['Sheet'] == 'Demand') & (ts['ID'] == comm)][year+5]) * 1e6 * profiles[comm]

In [ ]:
df['Demand'].head()

,Bayern.Space heating,Bayern.Process heating A,Bayern.Process heating B,Bayern.Process heating C,Bayern.Electric charging,Bayern.Elec_Rest
t,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20.211202,813.453207,488.071924,325.381283,263.388472,7293.329275
2,52.564630,813.593908,488.156345,325.437563,255.774458,7293.329275
3,255.384562,814.066925,488.440155,325.626770,252.108452,6979.555403
4,342.271702,815.270060,489.162036,326.108024,251.544451,6711.699659
